In [ ]:
from sklearn.impute import KNNImputer
import pandas as pd

imputer = KNNImputer(n_neighbors=5)
data_cleaned = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
print("Missing values after imputation:", data_cleaned.isnull().sum())

In [ ]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.05, random_state=42)
data_cleaned['Anomaly'] = iso.fit_predict(data_cleaned.drop(columns=['target']))
data_cleaned = data_cleaned[data_cleaned['Anomaly'] == 1].drop(columns=['Anomaly'])

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(data_cleaned.drop(columns=['target']), data_cleaned['target'])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, predictions))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
from flask import Flask, jsonify
from flask_socketio import SocketIO
import random, time

app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

def generate_data():
    while True:
        data = {
            "heart_rate": random.uniform(60, 100),
            "temperature": random.uniform(36, 40),
            "oxygen_level": random.uniform(95, 100)
        }
        socketio.emit("health_data", data)
        time.sleep(2)

socketio.start_background_task(generate_data)
socketio.run(app, port=5000, debug=True)

In [ ]:
<ResponsiveContainer width="100%" height={300}>
  <LineChart data={healthData}>
    <XAxis dataKey="time" />
    <YAxis />
    <Tooltip />
    <Line type="monotone" dataKey="heart_rate" stroke="red" />
    <Line type="monotone" dataKey="temperature" stroke="orange" />
    <Line type="monotone" dataKey="oxygen_level" stroke="blue" />
  </LineChart>
</ResponsiveContainer>